In [2]:
from pathlib import Path
import sys
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from PIL import Image
import cv2

ROOT = Path.cwd().resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))



In [3]:
# model

import torch
import torch.nn as nn
from torchvision import models

def build_resnet18(
    num_classes: int,
    pretrained: bool = True,
    dropout: float = 0.0,
    freeze_backbone: bool = False,
) -> nn.Module:

    try:
        weights = models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None
        model = models.resnet18(weights=weights)
    except Exception:
        model = models.resnet18(pretrained=pretrained)

    if freeze_backbone:
        for name, p in model.named_parameters():
            if not name.startswith("fc."):
                p.requires_grad = False

    in_feats = model.fc.in_features
    if dropout and dropout > 0.0:
        model.fc = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(in_feats, num_classes),
        )
    else:
        model.fc = nn.Linear(in_feats, num_classes)

    head = model.fc[-1] if isinstance(model.fc, nn.Sequential) else model.fc
    nn.init.kaiming_uniform_(head.weight, nonlinearity='relu')
    nn.init.zeros_(head.bias)

    return model


def save_checkpoint(model: nn.Module, path: str):
    torch.save(model.state_dict(), path)


def load_checkpoint(model: nn.Module, path: str, map_location='cpu', strict=True):
    state = torch.load(path, map_location=map_location)
    model.load_state_dict(state, strict=strict)
    return model

In [14]:
def homomorphic_filter_rgb(img_pil, sigma=30.0, gamma_l=0.7, gamma_h=1.5, eps=1e-6):
    img = np.array(img_pil.convert("RGB"), dtype=np.uint8)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb).astype(np.float32)
    luminance = ycrcb[..., 0] / 255.0
    log_y = np.log1p(luminance + eps)
    freq = np.fft.fftshift(np.fft.fft2(log_y))
    rows, cols = luminance.shape
    u = np.arange(rows) - rows / 2.0
    v = np.arange(cols) - cols / 2.0
    vv, uu = np.meshgrid(v, u)
    distance = np.sqrt(uu ** 2 + vv ** 2)
    high_pass = (gamma_h - gamma_l) * (1.0 - np.exp(-(distance ** 2) / (2.0 * (sigma ** 2)))) + gamma_l
    filtered = np.real(np.fft.ifft2(np.fft.ifftshift(freq * high_pass)))
    exp_y = np.expm1(filtered)
    exp_y = np.clip(exp_y, 0.0, None)
    normalized = cv2.normalize(exp_y.astype(np.float32), None, 0.0, 1.0, cv2.NORM_MINMAX)
    ycrcb[..., 0] = np.clip(normalized * 255.0, 0.0, 255.0)
    out = cv2.cvtColor(ycrcb.astype(np.uint8), cv2.COLOR_YCrCb2RGB)
    return Image.fromarray(out)

def logarithmic_enhancement_rgb(img_pil, c=1.0):
    img = np.array(img_pil.convert("RGB"), dtype=np.float32) / 255.0
    enhanced = c * np.log1p(img) / np.log1p(1.0)
    enhanced = np.clip(enhanced, 0.0, 1.0)
    enhanced = (enhanced * 255.0).astype(np.uint8)
    return Image.fromarray(enhanced)

def build_eval_transform(img_size, enhancement="homomorphic", homo_params=None, log_params=None):
    homo_params = homo_params or {}
    log_params = log_params or {}
    steps = [transforms.Resize((img_size, img_size))]
    if enhancement == "homomorphic":
        steps.append(transforms.Lambda(lambda im: homomorphic_filter_rgb(im, **homo_params)))
    elif enhancement == "log":
        steps.append(transforms.Lambda(lambda im: logarithmic_enhancement_rgb(im, **log_params)))
    elif enhancement == "homo+log":
        steps.append(transforms.Lambda(lambda im: homomorphic_filter_rgb(im, **homo_params)))
        steps.append(transforms.Lambda(lambda im: logarithmic_enhancement_rgb(im, **log_params)))
    elif enhancement == "none":
        pass
    steps.append(transforms.ToTensor())
    return transforms.Compose(steps)

In [5]:
def run_test(config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    target_classes = [c.strip() for c in config["class_order"].split(",") if c.strip()]
    class_to_idx = {c: i for i, c in enumerate(target_classes)}
    num_classes = len(target_classes)

    tf_eval = build_eval_transform(
        img_size=config["img_size"],
        enhancement=config["enhancement"],
        homo_params=config.get("homo_params"),
        log_params=config.get("log_params"),
    )

    full_test_ds = datasets.ImageFolder(config["test_dir"], transform=tf_eval)
    orig_classes = list(full_test_ds.classes)
    print("Raw test classes (from folder):", orig_classes)

    new_samples, new_targets = [], []
    for path, orig_label in full_test_ds.samples:
        cls_name = orig_classes[orig_label]
        if cls_name in class_to_idx:
            new_label = class_to_idx[cls_name]
            new_samples.append((path, new_label))
            new_targets.append(new_label)
    test_ds = full_test_ds
    test_ds.samples = new_samples
    test_ds.targets = new_targets
    test_ds.classes = target_classes
    test_ds.class_to_idx = class_to_idx
    print(f"Filtered test samples: {len(test_ds.samples)}")

    test_loader = DataLoader(
        test_ds,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=0,
        pin_memory=False,
    )

    model = build_resnet18(
        num_classes=num_classes,
        pretrained=False,
        dropout=0.0,
        freeze_backbone=False,
    ).to(device)
    load_checkpoint(model, config["model_path"], map_location=device)
    model.eval()
    print(f"Loaded model: {config['model_path']}")

    y_true, y_pred, y_prob_rows = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            logits = model(imgs)
            probs = torch.softmax(logits, dim=1).cpu().numpy()
            preds = logits.argmax(1).cpu().tolist()
            y_true += labels.tolist()
            y_pred += preds
            y_prob_rows += probs.tolist()

    y_prob = np.array(y_prob_rows)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    auc = roc_auc_score(y_true, y_prob, multi_class='ovr')

    print("\n===== TEST RESULTS (subset classes) =====")
    print("Class order:", target_classes)
    print(f"#Samples     = {len(y_true)}")
    print(f"Accuracy     = {acc:.4f}")
    print(f"F1 (macro)   = {f1:.4f}")
    print(f"AUC (OVR)    = {auc:.4f}")

    return {"acc": acc, "f1": f1, "auc": auc}


In [6]:
import os
import random
import argparse
from pathlib import Path
import sys

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.cuda import amp # For mixed precision training
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import numpy as np

from PIL import Image

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

parser = argparse.ArgumentParser()
parser.add_argument("--train_dir", type=str, default="/content/drive/MyDrive/Colab Notebooks/data/ECE_253/train")
parser.add_argument("--val_dir",   type=str, default="/content/drive/MyDrive/Colab Notebooks/data/ECE_253/validation")
parser.add_argument("--test_dir",  type=str, default="/content/drive/MyDrive/Colab Notebooks/data/ECE_253/test")
parser.add_argument("--img_size",  type=int, default=64)
parser.add_argument("--batch_size", type=int, default=128)
parser.add_argument("--epochs",     type=int, default=50)
parser.add_argument("--lr",         type=float, default=3e-4)
parser.add_argument("--weight_decay", type=float, default=1e-4)
parser.add_argument("--seed",       type=int, default=42)
parser.add_argument("--out_model",  type=str, default="/content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth")
parser.add_argument("--out_report", type=str, default="/content/drive/MyDrive/Colab Notebooks/data/ECE_253/test_results.txt")
parser.add_argument(
    "--keep_classes",
    type=str,
    default='beach,buildings,forest,harbor,freeway',

)

args = parser.parse_args([])

set_seed(args.seed)
IMG_SIZE = args.img_size

tf_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

tf_eval = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def subset_imagefolder(dataset, keep_class_names):

    orig_classes = list(dataset.classes)
    new_classes = list(keep_class_names)
    new_class_to_idx = {c: i for i, c in enumerate(new_classes)}

    new_samples = []
    new_targets = []

    for path, old_label in dataset.samples:
        cls_name = orig_classes[old_label]
        if cls_name in new_class_to_idx:
            new_label = new_class_to_idx[cls_name]
            new_samples.append((path, new_label))
            new_targets.append(new_label)

    dataset.samples = new_samples
    dataset.targets = new_targets
    dataset.classes = new_classes
    dataset.class_to_idx = new_class_to_idx

    return dataset
train_ds = datasets.ImageFolder(args.train_dir, transform=tf_train)
val_ds   = datasets.ImageFolder(args.val_dir,   transform=tf_eval)
test_ds  = datasets.ImageFolder(args.test_dir,  transform=tf_eval)


keep_cls_list = [c.strip() for c in args.keep_classes.split(",") if c.strip()]


train_ds = subset_imagefolder(train_ds, keep_cls_list)
val_ds   = subset_imagefolder(val_ds,   keep_cls_list)
test_ds  = subset_imagefolder(test_ds,  keep_cls_list)


num_classes = len(train_ds.classes)

train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
val_loader = DataLoader(
    val_ds,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
test_loader = DataLoader(
    test_ds,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

model = build_resnet18(
    num_classes=num_classes,
    pretrained=True,
    dropout=0.0,
    freeze_backbone=True,
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)
scaler = amp.GradScaler() # For mixed precision training

EPOCHS = args.epochs
best_val_acc = 0.0
epochs_no_improve = 0
patience = 10 # Number of epochs to wait for improvement before stopping

for epoch in range(1, EPOCHS + 1):
    model.train()
    loss_sum = 0.0
    y_true_train, y_pred_train = [], []

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [train]"):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        with amp.autocast(): # Mixed precision
            logits = model(imgs)
            loss = criterion(logits, labels)

        scaler.scale(loss).backward() # Scale loss and perform backward pass
        scaler.step(optimizer) # Update optimizer
        scaler.update() # Update scaler for next iteration

        loss_sum += loss.item()
        y_pred_train += logits.argmax(1).detach().cpu().tolist()
        y_true_train += labels.detach().cpu().tolist()

    train_loss = loss_sum / len(train_loader)
    train_acc = accuracy_score(y_true_train, y_pred_train)

    model.eval()
    val_true, val_pred = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS} [val]"):
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            with amp.autocast(): # Mixed precision for validation
                logits = model(imgs)
            preds = logits.argmax(1)

            val_pred += preds.cpu().tolist()
            val_true += labels.cpu().tolist()

    val_acc = accuracy_score(val_true, val_pred)
    scheduler.step(val_acc) # Step the learning rate scheduler

    print(f"Epoch {epoch}: train_loss={train_loss:.4f}  "
          f"train_acc={train_acc:.4f}  val_acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_checkpoint(model, args.out_model)
        epochs_no_improve = 0 # Reset counter if improvement
        print(f"  -> New best model saved to: {args.out_model} (val_acc={best_val_acc:.4f})")
    else:
        epochs_no_improve += 1
        print(f"  -> Val acc did not improve. Patience: {epochs_no_improve}/{patience}")
        if epochs_no_improve > patience:
            print(f"  -> Early stopping triggered after {patience} epochs without improvement.")
            break

best_model = build_resnet18(
    num_classes=num_classes,
    pretrained=False,
    dropout=0.0,
    freeze_backbone=False
).to(device)

load_checkpoint(best_model, args.out_model, map_location=device)
best_model.eval()

y_true, y_pred = [], []
y_prob_rows = []  # [N, C]

with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs = imgs.to(device, non_blocking=True)
        with amp.autocast(): # Mixed precision for testing
            logits = best_model(imgs)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        preds = logits.argmax(1).cpu().tolist()

        y_true += labels.tolist()
        y_pred += preds
        y_prob_rows += probs.tolist()

y_prob = np.array(y_prob_rows)

acc = accuracy_score(y_true, y_pred)
f1  = f1_score(y_true, y_pred, average='macro')
auc = roc_auc_score(y_true, y_prob, multi_class='ovr')

print(f"\nTest Results:")
print(f"  Acc      = {acc:.4f}")
print(f"  F1(macro)= {f1:.4f}")
print(f"  AUC(OVR) = {auc:.4f}")
with open(args.out_report, "w") as f:
    f.write("Test results (best model based on val_acc)\n")
    f.write(f"Accuracy      : {acc:.6f}\n")
    f.write(f"F1 (macro)    : {f1:.6f}\n")
    f.write(f"AUC (OVR)     : {auc:.6f}\n")
    f.write(f"Best val_acc  : {best_val_acc:.6f}\n")
    f.write(f"Num classes   : {num_classes}\n")
    f.write(f"Used classes  : {train_ds.classes}\n")
    f.write(f"Train dir     : {args.train_dir}\n")
    f.write(f"Val dir       : {args.val_dir}\n")
    f.write(f"Test dir      : {args.test_dir}\n")

print(f"\nTest results saved to: {args.out_report}")
print(f"Best model saved to:   {args.out_model}")


Using device: cuda


Epoch 1/50 [val]:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/50 [val]: 100%|██████████| 4/4 [00:46<00:00, 11.72s/it]


Epoch 1: train_loss=2.6537  train_acc=0.2686  val_acc=0.2220
  -> New best model saved to: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth (val_acc=0.2220)


Epoch 2/50 [train]:   0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 2/50 [val]:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if neces

Epoch 2: train_loss=2.0789  train_acc=0.3006  val_acc=0.2420
  -> New best model saved to: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth (val_acc=0.2420)


...

Epoch 49: train_loss=0.1410  train_acc=0.9686  val_acc=0.9380
  -> New best model saved to: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth (val_acc=0.9380)


Epoch 50/50 [train]:   0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 50/50 [val]:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if nec

Epoch 50: train_loss=0.1339  train_acc=0.9680  val_acc=0.9400
  -> New best model saved to: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth (val_acc=0.9400)


Testing:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Testing: 100%|██████████| 2/2 [00:45<00:00, 22.64s/it]


Test Results:
  Acc      = 0.9520
  F1(macro)= 0.9519
  AUC(OVR) = 0.9934

Test results saved to: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/test_results.txt
Best model saved to:   /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth


In [17]:
#baseline
config = {
    "test_dir": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/test",
    "model_path": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth",
    "img_size": 64,
    "batch_size": 128,
    "class_order": "beach,buildings,forest,harbor,freeway",
    "enhancement": "homomorphic",  # options: "homomorphic", "log","none"
    "homo_params": {"sigma": 30.0, "gamma_l": 0.7, "gamma_h": 1.5},
    "log_params": {"c": 1.0},
}

results = run_test(config)



Using device: cuda
Raw test classes (from folder): ['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt']
Filtered test samples: 250
Loaded model: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth

===== TEST RESULTS (subset classes) =====
Class order: ['beach', 'buildings', 'forest', 'harbor', 'freeway']
#Samples     = 250
Accuracy     = 0.9720
F1 (macro)   = 0.9720
AUC (OVR)    = 0.9940


In [6]:
config = {
    "test_dir": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/lowlight/test",
    "model_path": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth",
    "img_size": 64,
    "batch_size": 128,
    "class_order": "beach,buildings,forest,harbor,freeway",
    "enhancement": "none",  # options: "homomorphic", "log"
    "homo_params": {"sigma": 30.0, "gamma_l": 0.7, "gamma_h": 1.5},
    "log_params": {"c": 1.0},
}

results = run_test(config)



Using device: cuda
Raw test classes (from folder): ['beach', 'buildings', 'forest', 'freeway', 'harbor']
Filtered test samples: 250
Loaded model: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth

===== TEST RESULTS (subset classes) =====
Class order: ['beach', 'buildings', 'forest', 'harbor', 'freeway']
#Samples     = 250
Accuracy     = 0.7960
F1 (macro)   = 0.7957
AUC (OVR)    = 0.9482


In [12]:
config = {
    "test_dir": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/lowlight/test",
    "model_path": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth",
    "img_size": 64,
    "batch_size": 128,
    "class_order": "beach,buildings,forest,harbor,freeway",
    "enhancement": "homomorphic",  # options: "homomorphic", "log"
    "homo_params": {"sigma": 30.0, "gamma_l": 0.7, "gamma_h": 1.5},
    "log_params": {"c": 1.0},
}

results = run_test(config)



Using device: cuda
Raw test classes (from folder): ['beach', 'buildings', 'forest', 'freeway', 'harbor']
Filtered test samples: 250
Loaded model: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth

===== TEST RESULTS (subset classes) =====
Class order: ['beach', 'buildings', 'forest', 'harbor', 'freeway']
#Samples     = 250
Accuracy     = 0.8840
F1 (macro)   = 0.8830
AUC (OVR)    = 0.9800


In [13]:
config = {
    "test_dir": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/lowlight/test",
    "model_path": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth",
    "img_size": 64,
    "batch_size": 128,
    "class_order": "beach,buildings,forest,harbor,freeway",
    "enhancement": "log",  # options: "homomorphic", "log"
    "homo_params": {"sigma": 30.0, "gamma_l": 0.7, "gamma_h": 1.5},
    "log_params": {"c": 1.0},
}

results = run_test(config)

Using device: cuda
Raw test classes (from folder): ['beach', 'buildings', 'forest', 'freeway', 'harbor']
Filtered test samples: 250
Loaded model: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth

===== TEST RESULTS (subset classes) =====
Class order: ['beach', 'buildings', 'forest', 'harbor', 'freeway']
#Samples     = 250
Accuracy     = 0.8320
F1 (macro)   = 0.8346
AUC (OVR)    = 0.9707


In [20]:
config = {
    "test_dir": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/lowlight/test",
    "model_path": "/content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth",
    "img_size": 64,
    "batch_size": 128,
    "class_order": "beach,buildings,forest,harbor,freeway",
    "enhancement": "homo+log",  # options: "homomorphic", "log"
    "homo_params": {"sigma": 30.0, "gamma_l": 0.5, "gamma_h": 1.6},
    "log_params": {"c": 1.0},
}

results = run_test(config)

Using device: cuda
Raw test classes (from folder): ['beach', 'buildings', 'forest', 'freeway', 'harbor']
Filtered test samples: 250
Loaded model: /content/drive/MyDrive/Colab Notebooks/data/ECE_253/best_resnet18_rgb.pth

===== TEST RESULTS (subset classes) =====
Class order: ['beach', 'buildings', 'forest', 'harbor', 'freeway']
#Samples     = 250
Accuracy     = 0.8920
F1 (macro)   = 0.8911
AUC (OVR)    = 0.9852
